# Exploring the data

In [ ]:
#Get the log files

In [1]:
flumeFiles = ("/loudacre/weblogs/FlumeData.*")

In [2]:
accountFiles = ("/loudacre/accounts/part-m*")

In [22]:
baseStation = ("/loudacre/base_stations.tsv")

In [ ]:
# Setup RDS

In [27]:
accounts = sc.textFile(accountFiles).map(lambda line: line.split(','))

In [25]:
basestations = sc.textFile(baseStation).map(lambda line: line.split('\t'))

In [28]:
accounts.take(5)

[[u'1',
  u'2008-10-23 16:05:05.0',
  u'\\N',
  u'Donald',
  u'Becton',
  u'2275 Washburn Street',
  u'Oakland',
  u'CA',
  u'94660',
  u'5100032418',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'2',
  u'2008-11-12 03:00:01.0',
  u'\\N',
  u'Donna',
  u'Jones',
  u'3885 Elliott Street',
  u'San Francisco',
  u'CA',
  u'94171',
  u'4150835799',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'3',
  u'2008-12-21 09:19:50.0',
  u'\\N',
  u'Dorthy',
  u'Chalmers',
  u'4073 Whaley Lane',
  u'San Mateo',
  u'CA',
  u'94479',
  u'6506877757',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'4',
  u'2008-11-28 00:08:09.0',
  u'\\N',
  u'Leila',
  u'Spencer',
  u'1447 Ross Street',
  u'San Mateo',
  u'CA',
  u'94444',
  u'6503198619',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'5',
  u'2008-11-15 23:06:06.0',
  u'\\N',
  u'Anita',
  u'Laughlin',
  u'2767 Hill Street',
  u'Richmond',
  u'CA',
  u'94872',
  u'5107754354',
  u'2014-03-18 13:

In [26]:
basestations.take(5)

[[u'1', u'86502', u'Chambers', u'AZ', u'35.2375', u'-109.523'],
 [u'2', u'86514', u'Teec Nos Pos', u'AZ', u'36.7797', u'-109.359'],
 [u'3', u'85602', u'Benson', u'AZ', u'31.9883', u'-110.2941'],
 [u'4', u'86011', u'Flagstaff', u'AZ', u'35.6308', u'-112.0524'],
 [u'5', u'86016', u'Gray Mountain', u'AZ', u'35.6308', u'-112.0524']]

In [17]:
#How many users are there in total?
accounts.count()

129764

In [73]:
#Take cities from accounts rds
cities = accounts.map(lambda fields: (fields[6]))

In [78]:
#Display 5 cities
cities.take(5)

[u'Oakland', u'San Francisco', u'San Mateo', u'San Mateo', u'Richmond']

In [74]:
#Get unique cities
cities_unique = cities.distinct()

In [76]:
#How many cities have users
cities_unique.count()

56

In [83]:
#Get unique cities per basestations
cities_basestations = basestations.map(lambda fields: fields[2]).distinct()

In [84]:
#How many citeis have base stations
cities_basestations.count()

238

In [30]:
#How many basestation are there in total?
basestations.count()

377

# More detailed Analysis

In [100]:
#Get all accounts in the Oregon state
oregon = accounts.filter(lambda x: "OR" in x[7])

In [101]:
oregon.count()

13575

In [103]:
oregon.first()

[u'3509',
 u'2010-09-05 22:43:35.0',
 u'\\N',
 u'Cody',
 u'Martinez',
 u'265 Langtown Road',
 u'Eugene',
 u'OR',
 u'97487',
 u'5414866653',
 u'2014-03-18 13:29:53.0',
 u'2014-03-18 13:29:53.0']

In [110]:
oregonUsers = oregon.map(lambda fields: (fields[6], fields[0]))

In [112]:
oregonUsers.first()

(u'Eugene', u'3509')

In [129]:
userCityFreq = oregonUsers.countByKey()

In [131]:
#Output the number users in each city in Orgeon
(userCityFreq)

defaultdict(<type 'int'>, {u'Pendleton': 1455, u'Salem': 1496, u'Bend': 1528, u'Portland': 4602, u'Klamath Falls': 1463, u'Medford': 1511, u'Eugene': 1520})

In [337]:
#Get RDD of all users and their cities
allUsersCities = accounts.map(lambda fields: (fields[6])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [338]:
#Show the first 5  rows
allUsersCities.take(5)

[(u'Sacramento', 6820),
 (u'San Diego', 4618),
 (u'Santa Barbara', 3067),
 (u'Anaheim', 1544),
 (u'Salem', 1496)]

In [242]:
allUsersCitiesFreq = allUsersCities.countByKey()

In [243]:
allUsersCitiesFreq

defaultdict(<type 'int'>, {u'Show Low': 1, u'Palm Springs': 1, u'North Hollywood': 1, u'Inglewood': 1, u'Pendleton': 1, u'Santa Monica': 1, u'Elko': 1, u'Richmond': 1, u'Prescott': 1, u'Ely': 1, u'Redding': 1, u'Reno': 1, u'Santa Rosa': 1, u'Santa Ana': 1, u'San Francisco': 1, u'Riverside': 1, u'San Mateo': 1, u'Torrance': 1, u'Eugene': 1, u'Flagstaff': 1, u'Sacramento': 1, u'San Diego': 1, u'Alhambra': 1, u'Las Vegas': 1, u'Marysville': 1, u'Berkeley': 1, u'Van Nuys': 1, u'Klamath Falls': 1, u'Carson City': 1, u'Bakersfield': 1, u'Los Angeles': 1, u'Pasadena': 1, u'Industry': 1, u'San Jose': 1, u'Burbank': 1, u'Eureka': 1, u'Anaheim': 1, u'Phoenix': 1, u'Palo Alto': 1, u'Glendale': 1, u'San Bernardino': 1, u'Medford': 1, u'Mojave': 1, u'Stockton': 1, u'Fresno': 1, u'Globe': 1, u'Tucson': 1, u'Oxnard': 1, u'Long Beach': 1, u'Salinas': 1, u'Salem': 1, u'Bend': 1, u'Portland': 1, u'Oakland': 1, u'Kingman': 1, u'Santa Barbara': 1})

In [333]:
allBasesCities = basestations.map(lambda fields: (fields[2])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [335]:
allBasesCities.take(100)

[(u'Blue', 1),
 (u'Loomis', 1),
 (u'Baker City', 1),
 (u'Packwood', 1),
 (u'Bullhead City', 1),
 (u'Santa Fe Springs', 1),
 (u'Round Mountain', 1),
 (u'Apple Valley', 1),
 (u'Eldridge', 1),
 (u'Santa Rosa', 1),
 (u'Toledo', 1),
 (u'San Francisco', 15),
 (u'Hansen', 1),
 (u'Henderson', 1),
 (u'Sterling', 1),
 (u'Lodi', 1),
 (u'North Highlands', 1),
 (u'Corcoran', 1),
 (u'Sacramento', 4),
 (u'Scottsdale', 2),
 (u'Laton', 1),
 (u'Needles', 1),
 (u'Butte Falls', 1),
 (u'Berkeley', 1),
 (u'Oregon House', 1),
 (u'Othello', 1),
 (u'Upland', 1),
 (u'Mc Call', 1),
 (u'Canyon Country', 1),
 (u'Whittier', 3),
 (u'Greenwood', 1),
 (u'Blue Gap', 1),
 (u'Flagstaff', 1),
 (u'Rimrock', 1),
 (u'Laveen', 1),
 (u'King City', 1),
 (u'Ballico', 1),
 (u'Brea', 1),
 (u'Nephi', 1),
 (u'Blanchard', 1),
 (u'Saint Benedict', 1),
 (u'San Jacinto', 1),
 (u'Redding', 1),
 (u'Chico', 1),
 (u'Olympia', 1),
 (u'Bellingham', 1),
 (u'Witter Springs', 1),
 (u'Fresno', 2),
 (u'Glenoma', 1),
 (u'Dillard', 1),
 (u'Belmont',

In [339]:
cityUsersBases = allUsersCities.join(allBasesCities)

In [340]:
cityUsersBases.take(10)

[(u'Carson City', (1360, 1)),
 (u'Glendale', (1493, 2)),
 (u'Richmond', (1746, 1)),
 (u'San Francisco', (3354, 15)),
 (u'Sacramento', (6820, 4)),
 (u'Flagstaff', (1181, 1)),
 (u'Long Beach', (4500, 3)),
 (u'Palm Springs', (1482, 1)),
 (u'Redding', (1587, 1)),
 (u'Riverside', (1477, 1))]